In [1]:
import pandas as pd 
import numpy as np 
import joblib

In [2]:


def concat(file,numbers):
    urls = []
    for num in numbers:         
        url=f"data/yahoo_ipo/{file}{num}"
        urls.append(url)
        
    files = []
    for url in urls:
        file = pd.read_csv(url)
        files.append(file)
    df = pd.concat(files)
    
    return df
 

In [3]:
numbers1 = ['200','1000','2000','2500',
           '2_500','2_1000','2_1500','2_2000','2_2500','2_3500','2_4000','2_4500','2_5000','2_5500',
           '2_6000','2_6500','2_7000','2_8000','2_8500','2_9000','2_10000']
numbers180 = ['1000','2000','3000','3500','4000','4500','5000','5500','6000','6500','7000','7500','8000','8500',
              '9000','9500','10000']

file1='yf2_'
file180='post_6mo_'


ipo_1_raw = concat(file1,numbers1)
ipo_180_raw = concat(file180,numbers180)


In [4]:
def ipo_returns(df,col1,col2):
    ret = (df[col2] - df[col1] )/df[col1]
    return ret

In [5]:
ipo_1_raw['day_1_%'] = ipo_returns(ipo_1_raw,'open','close')

In [6]:
ipo_180_raw['month_7_%'] = ipo_returns(ipo_180_raw,'open','close')

In [7]:
dfs = [ipo_1_raw,ipo_180_raw]
for df in dfs:
    df.drop(columns=['Unnamed: 0','date'],inplace=True)
    df.rename(columns = {'formatted_date': 'trade_date'},
              inplace=True)
    df.drop_duplicates(inplace=True)
    df.groupby(['ticker']).max().reset_index()
    df['trade_date'] = pd.to_datetime(df['trade_date'], errors = 'coerce')
    
    


In [8]:
yf2 = ipo_1_raw.merge(ipo_180_raw, how ='left', on='ticker')

In [9]:
yf2.rename(columns={'trade_date_x': 'trade_date'},
                inplace=True) 


In [10]:

yf2= yf2[['ticker','trade_date','day_1_%','month_7_%']]
yf2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3420 entries, 0 to 3419
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ticker      3420 non-null   object        
 1   trade_date  3420 non-null   datetime64[ns]
 2   day_1_%     3420 non-null   float64       
 3   month_7_%   2588 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 133.6+ KB


In [11]:
joblib.dump(yf2,"yf2.pkl")

['yf2.pkl']